# How to Connect to Coinbase Pro and Display Advanced Crypto Currency Trading Indicators?

by Nils Ackermann

## Introduction

In case you want to get started as a crypto investor and you want to take matters into your own hands, it is important that you are able to connect to a currency exchange such as [pro.coinbase.com](https://pro.coinbase.com) to pull market data. In this notebook you will learn the following:

- How to conveniently connect to coinbase pro via the cbpro Python library
- How to load data from coinbase pro into a pandas dataframe
- How to transform crypto currency data and add additional trading indicators (Simple Moving Average, Exponential Moving Average, MACD, MACD Signal, MACD Histogram) with the help of pandas
- How to visualize crypto currency market data via interactive charts using plotly

## Step 0 - Import Libraries and Set Constants

We will use very basic python libraries. The only "special" one that you might not be aware of is [cbpro](https://github.com/danpaquin/coinbasepro-python). This is a library which allows you to conveniently connect to the Coinbase Pro API. Before we connect, we will set a few parameters like the base currency (e.g. USD or EUR) that we want to use as well as a list of cryptos that we want to invest in. Feel free to modify the list with your cryptos. 

In [ ]:
import cbpro
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from math import trunc
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Choose currency that should be used to display data
FIAT_CURRENCIES = ['EUR','USD']
MY_BASE_CURRENCY = FIAT_CURRENCIES[0]
# I am only interested in a few currencies that I want to trade, so let's add them here:
MY_CRYPTO_CURRENCIES = ["BTC","ETH","LTC","ALGO","SHIB","MANA"] 

RED_COLOR = '#D61170'
GREEN_COLOR = '#3FFA68'
MACD_COLOR = '#A9B1F5'
MACD_SIGNAL_COLOR = '#FFD1A3'
CANDLE_INCREASE_COLOR = 'cyan'
CANDLE_DECREASE_COLOR = 'gray'
#PLOTLY_TEMPLATE = 'plotly_dark'
PLOTLY_TEMPLATE = 'plotly'
GRANULARITIES = ['daily','60min','15min','1min']

## Step 1 - Establish Connection to Coinbase Pro

Connecting to the Coinbase API is super simple with the help of the cbpro library. For this notebook, we only need to connect to the public Coinbase Pro API, therefore no authentication is needed.

In [ ]:
# Initiate public Coinbase Pro client
public_client = cbpro.PublicClient()
server_time = public_client.get_time()
# Server time does not comply to iso format, therefore slight modification of string needed
server_time_now = datetime.fromisoformat(server_time['iso'].replace('T', ' ', 1)[0:19])
print(server_time_now)
print(datetime.now())

## Step 2 - Show Available Products (Crypto Currencies)

First, we will review some basic information about the crypto currencies that we chose to trade on Coinbase Pro.

In [ ]:
df_currencies = pd.DataFrame(public_client.get_products())
print("\nNumber of columns in the dataframe: %i" % (df_currencies.shape[1]))
print("Number of rows in the dataframe: %i\n" % (df_currencies.shape[0]))
columns = list(df_currencies.columns)
df_currencies[df_currencies.id.isin(list(map(lambda orig_string: orig_string+'-'+MY_BASE_CURRENCY, MY_CRYPTO_CURRENCIES)))][['id', 'quote_currency', 'base_min_size', 'base_max_size']]

In case you later want to place orders via the Coinbase Pro API (not covered in this notebook), it is important to provide the right level of precision for the order volume. So let's calculate the allowed digits after the decimal point for each of our currencies:

In [ ]:
decimal_info_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    product_id = currency+'-'+MY_BASE_CURRENCY
    min_size = float(df_currencies.query('id == @product_id')['base_min_size'])
    precision = min_size-trunc(min_size)
    if precision == 0:
        decimals = 0
    elif 1 > precision > 0.1:
        decimals = 1
    elif 0.1 > precision > 0.01:
        decimals = 2
    elif 0.01 > precision > 0.001:
        decimals = 3
    elif 0.001 > precision > 0.0001:
        decimals = 4
    else:
        decimals = 5
    row_data = [product_id,min_size,precision,decimals]
    decimal_info_rows.append(row_data)
df_decimal_info = pd.DataFrame(decimal_info_rows, columns = ['id','base_min_size','precision','allowed_decimals'])
df_decimal_info

## Step 3 - Load 24h Stats

Next, we will review the basic stats from the last 24 hours for each of the crypto currencies that we have defined in our list above. We will also add a custom column "performance" which will show us the performance from the beginning to the end of the reporting period. We will also do some nice formatting of the numbers for printing purposes.

In [ ]:
currency_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    data = public_client.get_product_24hr_stats(currency+'-'+MY_BASE_CURRENCY)
    currency_rows.append(data)
df_24hstats = pd.DataFrame(currency_rows, index = MY_CRYPTO_CURRENCIES)
df_24hstats['currency'] = df_24hstats.index
df_24hstats['open'] = df_24hstats['open'].astype(float)
df_24hstats['high'] = df_24hstats['high'].astype(float)
df_24hstats['low'] = df_24hstats['low'].astype(float)
df_24hstats['volume'] = df_24hstats['volume'].astype(float)
df_24hstats['last'] = df_24hstats['last'].astype(float)
df_24hstats['volume_30day'] = df_24hstats['volume_30day'].astype(float)
df_24hstats['performance'] = ((df_24hstats['last']-df_24hstats['open']) / df_24hstats['open']) * 100
df_24hstats_formatted = df_24hstats.copy()
df_24hstats_formatted['performance'] = df_24hstats_formatted['performance'].apply(lambda x: "{:.2f}%".format((x)))
df_24hstats_formatted['open'] = df_24hstats_formatted['open'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['high'] = df_24hstats_formatted['high'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['low'] = df_24hstats_formatted['low'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['last'] = df_24hstats_formatted['last'].apply(lambda x: "{:,.2f}€".format((x)))
print('Performance within last 24 hours in ' + MY_BASE_CURRENCY + ':')
df_24hstats_formatted.style.hide_columns(['volume','volume_30day'])

## Step 4 - Load Historic Data

Above, we have only looked at the last 24 hours. Now we will download data using four different time slices and periods:
- Daily data of the last 90 days
- Hourly data of the last 300 hours
- 15-minute data of the last 75 hours
- 1-minute data of the last 2 hours

All this data is stored in a pandas dataframe called df_history.

In [ ]:
currency_history_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    # Daily data:
    start_date = (server_time_now - timedelta(days=90)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=86400)
    [x.append(currency) for x in data]
    [x.append('daily') for x in data]
    currency_history_rows.extend(data)
    # 60 minutes data:
    start_date = (server_time_now - timedelta(hours=300)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=3600)
    [x.append(currency) for x in data]
    [x.append('60min') for x in data]
    currency_history_rows.extend(data)
    # 15 minutes data:
    start_date = (server_time_now - timedelta(hours=75)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=900)
    [x.append(currency) for x in data]
    [x.append('15min') for x in data]
    currency_history_rows.extend(data)
    # 1 minutes data:
    start_date = (server_time_now - timedelta(hours=2)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=60)
    [x.append(currency) for x in data]
    [x.append('1min') for x in data]
    currency_history_rows.extend(data)
df_history = pd.DataFrame(currency_history_rows)
# Add column names in line with the Coinbase Pro documentation
df_history.columns = ['time','low','high','open','close','volume','currency','granularity']
# We will add a few more columns just for better readability
df_history['date'] = pd.to_datetime(df_history['time'], unit='s')
df_history['year'] = pd.DatetimeIndex(df_history['date']).year
df_history['month'] = pd.DatetimeIndex(df_history['date']).month
df_history['day'] = pd.DatetimeIndex(df_history['date']).day
df_history['hour'] = pd.DatetimeIndex(df_history['date']).hour
df_history['minute'] = pd.DatetimeIndex(df_history['date']).minute

In [ ]:
print(f'Daily data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'daily\'').head(5).style.hide_index().hide_columns(['time','date','hour','minute'])

In [ ]:
print(f'60 minute data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'60min\'').tail(5).style.hide_index().hide_columns(['time','minute'])

In [ ]:
print(f'15 minute data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'15min\'').tail(5).style.hide_index().hide_columns(['time'])

In [ ]:
print(f'1 minute data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'1min\'').tail(5).style.hide_index().hide_columns(['time'])

## Step 5 - Get More Insights

Let's add a few more parameters to the basic data (residing in df_history):

- **SMA3 and SMA7 and SMA12** (Simple Moving Average over last 3 and 7 time slices): This is a price based, lagging (or reactive) indicator that displays the average price of a security over a certain period of time. Moving Averages smooth out the “noise” when interpreting charts. Noise is made up of fluctuations of both price and volume. Because a Moving Average is a lagging indicator and reacts to events that have already happened, it is not used as a predictive indicator but rather an interpretive one. Simple Moving Average is an unweighted Moving Average. This means that each period in the data set has equal importance and is weighted equally. 
- **EMA12 and EMA26** (Exponential Moving Average over last 12 and 26 time slices): The EMA has much less lag than the SMA because it puts a greater importance on more recent prices. It therefore turns quicker than the SMA. You can choose different time slices, but 12 and 26 are commonly used durations.
- **MACD** (Moving Average Convergence/Divergence): A great indicator to identify the overall trend of any security (e.g. crypto currencies). It takes the difference between the short term EMA and the long term EMA. A positive MACD value is an indicator for a positive market trend. A negative MACD value is an indicator for a negative market trend. 
- **MACD Signal**: The MACD Signal line is the EMA over a certain set of time slices of the MACD line. Typically, the value is set to 9 time slices. 
- **MACD Histogram**: It is the difference between the MACD line and the MACD Signal line. A bullish crossover occurs when the MACD line crosses above the MACD Signal line. A bearish crossover occurs if the MACD line crosses below the MACD Signal line. 
- **Open to close performance**: For each time slice, the difference between the closing price and the opening price of that particular period is given as a percentage.
- **High low difference**: The deviation within the current period between the highest and lowest price as a percentage. This shows the volatility within one time slice.
- **Current high versus previous close absolute value**: The absolute value of the difference between the high of the current period and the close value of the previous period (needed for ATR calculation)
- **Current low versus previous close absolute value**: The absolute value of the difference between the low of the current period and the close value of the previous period (needed for ATR calculation)
- **True Range**: The max value of the previous three values (needed for ATR calculation).
- **Average True Range (ATR)**:  This is a is a market volatility indicator used in technical analysis. It takes the moving average of the True Range (typically over a 14 day period). It is not (!) an indicator for price direction. 
- **Absolute performance of last 3 periods**: This indicator provides the performance over the last three time slices as an absolute value.
- **Performance of last 3 periods in percentage**: This indicator provides the performance over the last three time slices as a relative value.
- **Bull or Bear market**: It simply writes "Bull" in case the MACD Histogram is positive and it writes "Bear" in case the MACD Histogram value is negative. 
- **Market trend continued**: To identify a shift from Bull to Bear market and vice versa, this column holds "True" in case the trend from the previous time slice continues and "False" in case a trend change has occurred. 

The enhanced data is stored in a pandas dataframe called df_history_enhanced. We will again use the four different granularities defined above.

In case you want to learn more about the different trading indicators like SMA, EMA, MACD and ATR, take a look [here](https://www.tradingview.com) or [here](https://www.investopedia.com/technical-analysis-4689657).

In [ ]:
AVERAGE_TRUE_RANGE_PERIODS = 14
currency_history_rows_enhanced = []
for currency in MY_CRYPTO_CURRENCIES:
    for granularity in GRANULARITIES:
        df_history_currency = df_history.query('granularity == @granularity & currency == @currency').copy()
        # Oldest to newest date sorting needed for SMA calculation
        df_history_currency = df_history_currency.sort_values(['date'], ascending=True)
        df_history_currency['SMA3'] = df_history_currency['close'].rolling(window=3).mean()
        df_history_currency['SMA7'] = df_history_currency['close'].rolling(window=7).mean()
        df_history_currency['EMA12'] = df_history_currency['close'].ewm(span=12, adjust=False).mean()
        df_history_currency['EMA26'] = df_history_currency['close'].ewm(span=26, adjust=False).mean()
        df_history_currency['MACD'] = df_history_currency['EMA12'] - df_history_currency['EMA26']
        df_history_currency['MACD_signal'] = df_history_currency['MACD'].ewm(span=9, adjust=False).mean()
        df_history_currency['macd_histogram'] = ((df_history_currency['MACD']-df_history_currency['MACD_signal']))
        df_history_currency['open_to_close_perf'] = ((df_history_currency['close']-df_history_currency['open']) / df_history_currency['open'])
        df_history_currency['high_low_diff'] = (df_history_currency['high']-df_history_currency['low'])
        df_history_currency['high_prev_close_abs'] = np.abs(df_history_currency['high'] - df_history_currency['close'].shift())
        df_history_currency['low_prev_close_abs'] = np.abs(df_history_currency['low'] - df_history_currency['close'].shift())
        df_history_currency['true_range'] = df_history_currency[['high_low_diff', 'high_prev_close_abs', 'low_prev_close_abs']].max(axis=1)
        #df_history_currency['average_true_range'] = df_history_currency['true_range'].rolling(window=14).sum()/14
        df_history_currency['average_true_range'] = df_history_currency['true_range'].ewm(alpha=1/AVERAGE_TRUE_RANGE_PERIODS, min_periods=AVERAGE_TRUE_RANGE_PERIODS, adjust=False).mean()
        df_history_currency['high_low_perf'] = ((df_history_currency['high']-df_history_currency['low']) / df_history_currency['high'])
        df_history_currency['open_perf_last_3_period_abs'] = df_history_currency['open'].rolling(window=4).apply(lambda x: x.iloc[1] - x.iloc[0])
        df_history_currency['open_perf_last_3_period_per'] = df_history_currency['open'].rolling(window=4).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
        df_history_currency['bull_bear'] = np.where(df_history_currency['macd_histogram']< 0, 'Bear', 'Bull')
        currency_history_rows_enhanced.append(df_history_currency)
df_history_enhanced = pd.concat(currency_history_rows_enhanced, ignore_index=True)
# Last step to tag changes in market trends from one period to the other (sorting important)
df_history_enhanced = df_history_enhanced.sort_values(['currency','granularity','date'], ascending=True)
df_history_enhanced['market_trend_continued'] = df_history_enhanced.bull_bear.eq(df_history_enhanced.bull_bear.shift()) & df_history_enhanced.currency.eq(df_history_enhanced.currency.shift()) & df_history_enhanced.granularity.eq(df_history_enhanced.granularity.shift())
df_history_enhanced.head(10)

## Step 6 - Print Out Selected Information

Now, we can leverage the data which is available in df_history_enhanced to compile some basic parameters that we might want to use in a personal dashboard down the line.

In [ ]:
additional_info_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True) #oldest to newest
    row_data = [currency,
        float(df_24hstats.query('currency == @currency')['last'].to_string(index=False)), # current value
        df_history_enhanced.query('granularity == \'1min\' and currency == @currency').tail(1)['bull_bear'].to_string(index=False), # 1min market trend
        df_history_enhanced.query('granularity == \'1min\' and currency == @currency').iloc[-2]['bull_bear'], # prev 1min market trend
        df_history_enhanced.query('granularity == \'15min\' and currency == @currency').tail(1)['bull_bear'].to_string(index=False), #15min market trend
        df_history_enhanced.query('granularity == \'15min\' and currency == @currency').iloc[-2]['bull_bear'], # prev 15min market trend
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').tail(1)['bull_bear'].to_string(index=False), # 60min market trend
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').iloc[-2]['bull_bear'], # prev 60min market trend
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').tail(24)['low'].min(), # lowest last 24h
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(30)['low'].min(), # lowest last 30d
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(90)['low'].min(), # lowest last 90d
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').tail(24)['high'].max(), # highest last 24h
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(30)['high'].max(), # highest last 30d
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(90)['high'].max(), # highest last 90d
        round(df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(90)['high_low_perf'].max(),3), # max daily high/low difference last 90d
        round(df_history_enhanced.query('granularity == \'60min\' and currency == @currency')['high_low_perf'].max(),3) # max daily high/low difference last 12.5d
        ]
    additional_info_rows.append(row_data)
df_additional_info = pd.DataFrame(additional_info_rows, columns = ['currency',
                                                                    'current_val',
                                                                    '1min_trend',
                                                                    '1min_prev_trend',
                                                                    '15min_trend',
                                                                    '15min_prev_trend',
                                                                    '60min_trend',
                                                                    '60min_prev_trend',
                                                                    '24h_low',
                                                                    '30day_low',
                                                                    '90day_low',
                                                                    '24h_high',
                                                                    '30day_high',
                                                                    '90day_high',
                                                                    '90day_max_high_low_span',
                                                                    '12day_max_high_low_span'])
df_additional_info['current_val'] = df_additional_info['current_val'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['24h_low'] = df_additional_info['24h_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['30day_low'] = df_additional_info['30day_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_low'] = df_additional_info['90day_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['24h_high'] = df_additional_info['24h_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['30day_high'] = df_additional_info['30day_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_high'] = df_additional_info['90day_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_max_high_low_span'] = (df_additional_info['90day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info['12day_max_high_low_span'] = (df_additional_info['12day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info.set_index('currency').transpose()

## Step 7 - Visualize Data

For visualization, we will use the powerful [plotly](https://plotly.com) library. It will give us the possibility to slice and dice data in an interactive way. Exactly what we need for an in-depth analysis of our daily market data.

### Simple Visualization

First, let's create a simple OHLC (Open, High, Low, Close) candle stick chart for each of our currencies.

In [ ]:
df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True) #oldest to newest for visualization
granularity = '60min'
for currency in MY_CRYPTO_CURRENCIES:
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and currency == \'{currency}\'')
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        name='OHLC'))
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['average_true_range'],
        line=dict(color=MACD_COLOR),
        mode='lines',
        name="ATR",
        yaxis="y2"
    ))
    cs = fig.data[0]
    cs.increasing.fillcolor = CANDLE_INCREASE_COLOR
    cs.increasing.line.color = CANDLE_INCREASE_COLOR
    cs.decreasing.fillcolor = CANDLE_DECREASE_COLOR
    cs.decreasing.line.color = CANDLE_DECREASE_COLOR
    fig.update_layout(
        yaxis=dict(
            title=MY_BASE_CURRENCY,
            side="right",
        ),
        yaxis2=dict(
            title="Averate True Range (Volatility)",
            anchor="free",
            overlaying="y",
            side="left",
        ),
        yaxis2_showgrid = False,
        yaxis2_zeroline = False,
        xaxis_rangeslider_visible=False,
        title=f'OHLC Chart for {currency} with time interval {granularity}',
        hovermode='x unified',
        xaxis=dict(
            rangeslider=dict(visible=True),
            type="date"
        ),
        template = PLOTLY_TEMPLATE,)
    fig.show()

### Advanced Visualization

Now that we have drawn a basic OHLC chart, let's get a bit more fancy. We will now draw the chart again. But this time we will add the following information:
- MACD line
- MACD signal line
- Colored area to show if we are currently in a bull market (green) or bear market (red) in each given period

Before we do that, there is one more important step to take: we must construct the data needed to color different periods for the market trends (bull versus bear). Therefore, let's create another pandas dataframe df_history_market_trend_intervals.

In [ ]:
market_trend_interval_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    for granularity in GRANULARITIES:
        df_history_market_trend_intervals = df_history_enhanced.query('currency == @currency and market_trend_continued == False and granularity == @granularity').copy()
        #df_history_market_trend_intervals['previous_period_date'] = df_history_market_trend_intervals.date.shift()
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals.date.shift(-1)
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals['next_period_date']
        df_history_market_trend_intervals.next_period_date = df_history_market_trend_intervals.next_period_date.fillna(datetime.now())
        df_history_market_trend_intervals['color'] = df_history_market_trend_intervals['bull_bear'].apply(lambda x: GREEN_COLOR if x == 'Bull' else RED_COLOR)
        df_history_market_trend_intervals = df_history_market_trend_intervals[['currency','granularity','bull_bear','color','date','next_period_date']].rename(columns={"date": "start_date", "next_period_date": "finish_date"})
        market_trend_interval_rows.append(df_history_market_trend_intervals)
df_history_market_trend_intervals = pd.concat(market_trend_interval_rows, ignore_index=True)

In [ ]:
granularity = '60min'
for currency in MY_CRYPTO_CURRENCIES:
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and currency == \'{currency}\'').copy()
    df_history_market_trend_for_chart = df_history_market_trend_intervals.query(f'granularity == \'{granularity}\' and currency == \'{currency}\'').copy()
    macd_min = df_history_for_chart['MACD'].min()
    macd_max = df_history_for_chart['MACD'].max()
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Ohlc(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        increasing_line_color= CANDLE_INCREASE_COLOR,
        decreasing_line_color= CANDLE_DECREASE_COLOR,
        name='OHLC'),
        secondary_y=True),
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD'],
        name='MACD',
        line=dict(color=MACD_COLOR),
        mode='lines'),
        secondary_y=False)
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD_signal'],
        name='MACD Signal',
        line=dict(color=MACD_SIGNAL_COLOR),
        mode='lines'),
        secondary_y=False)
    for i in range(df_history_market_trend_for_chart['currency'].count()):
        fig.add_vrect(x0=df_history_market_trend_for_chart.iloc[i]['start_date'],
            x1=df_history_market_trend_for_chart.iloc[i]['finish_date'],
            col=1,
            #fillcolor=df_history_market_trend_for_chart.iloc[i]['color'],
            line=dict(color=df_history_market_trend_for_chart.iloc[i]['color']),
            opacity=1.0,
            line_width=1)
    fig.update_layout(
        title=f'OHLC Chart for {currency} with time interval {granularity}',
        hovermode='x unified',
        legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.94),
        #autosize=False,
        #width=900,
        #height=600,
        template = PLOTLY_TEMPLATE,
        yaxis_range=[macd_min,macd_max],
        yaxis1_showgrid = False,
        yaxis1_showticklabels = False,
        yaxis1_zeroline = False,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    if granularity == 'daily':
        fig.update_layout(xaxis_tickformat = '%d %b (%a)')
    else:
        fig.update_layout(xaxis_tickformat = '%d %b %H:%M')
    fig.show()